## Import requirements

In [1]:
import yaml, time, json, traceback
from collections import defaultdict
from typing import Dict, List
from pprint import PrettyPrinter
from tabulate import tabulate
import networkx as nx
from itertools import count
from fabrictestbed_extensions.fablib.fablib import fablib
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

#### Define a better printer for debugging process

In [2]:
def pretty_print_defaultdict(d):
    # Create PrettyPrinter instance with custom formatting
    pp = PrettyPrinter(indent=4, width=80, sort_dicts=True)
    
    # Convert defaultdict to dict before printing
    pp.pprint(dict(d))

## Get values from YAML file

In [3]:
with open('./config20.yaml', 'r') as config_file:
    base_config = yaml.safe_load(config_file)
    
client_node_names: List[str] = base_config['topology']['client_nodes']
client_connections: Dict[str, List[str]] = base_config['topology']['client_connections']
general_node_names: List[str] = base_config['topology']['nodes']
general_connections: Dict[str, List[str]] = base_config['topology']['connections']

all_nodes = client_node_names + general_node_names
node_amount = len(all_nodes)

print(f'Client node names: {client_node_names}')
print(f'\nClient connections: {json.dumps(client_connections, indent=2)}')
print(f'\nGeneral node names: {general_node_names}')
print(f'\nGeneral connections: {json.dumps(general_connections, indent=2)}')

Client node names: ['node01', 'node06', 'node11']

Client connections: {
  "node01": [
    "node02"
  ],
  "node06": [
    "node07"
  ],
  "node11": [
    "node12"
  ]
}

General node names: ['node02', 'node03', 'node04', 'node05', 'node07', 'node08', 'node09', 'node10', 'node12', 'node13', 'node14', 'node15', 'node16', 'node17', 'node18', 'node19', 'node20']

General connections: {
  "node02": [
    "node03",
    "node04"
  ],
  "node03": [
    "node02",
    "node05",
    "node07"
  ],
  "node04": [
    "node02",
    "node08",
    "node09"
  ],
  "node05": [
    "node03",
    "node10"
  ],
  "node07": [
    "node03",
    "node08",
    "node13"
  ],
  "node08": [
    "node04",
    "node07",
    "node14"
  ],
  "node09": [
    "node04",
    "node10",
    "node15"
  ],
  "node10": [
    "node05",
    "node09",
    "node16"
  ],
  "node12": [
    "node13",
    "node16"
  ],
  "node13": [
    "node07",
    "node12",
    "node17"
  ],
  "node14": [
    "node08",
    "node15",
    "node18"
 

## Define global variables

In [4]:
manager = fablib_manager()

time_stamp = int(time.time())
experiment_slice = fablib.new_slice(name=f"hydra_auto_{time_stamp}")

# Basic node configs
experiment_cores = 4
experiment_ram = 8
experiment_disk = 20
experiment_image = "default_ubuntu_20"

# In case of fail, pick manually
def filter_sites(site):    
    try:
        if (site["state"] == "Active" and 
            site["hosts"] > 0 and 
            site["cores_available"] >= experiment_cores * 2 and 
            site["ram_available"] >= experiment_ram * 2 and 
            site["disk_available"] >= experiment_disk * 2
        ):
            return True
        return False
    except Exception as error:
        print(f"Filter error for site {site}: {error}")
        return False

# Change amount of site based on the topology size
experiment_sites = manager.get_random_sites(
    count=5,
    avoid=['NEWY', 'UCSD', 'EDUKY'],
    filter_function=filter_sites,
    update=True,
    unique=True 
)

print(f'Available sites: {experiment_sites}')

Available sites: ['DALL', 'HAWI', 'NCSA', 'WASH', 'MICH']


## Add nodes and interfaces

In [5]:
site_selector = count()

nodes = defaultdict(dict)
nodes_interfaces = defaultdict(list)
client_nodes = defaultdict(dict)
client_interfaces = defaultdict(list)
networks = defaultdict(list)

# Helper function to count total connections for a node
def count_total_connections(node_name):
    # For client nodes, just count their outgoing connections
    if node_name in client_node_names:
        return len(client_connections.get(node_name, []))
    
    # For general nodes, count unique connections
    connections = set()
    
    # Add outgoing connections
    for target in general_connections.get(node_name, []):
        connections.add(tuple(sorted([node_name, target])))
    
    # Add incoming connections
    for source, targets in general_connections.items():
        if node_name in targets:
            connections.add(tuple(sorted([source, node_name])))
    
    # Add connections from clients
    for client, targets in client_connections.items():
        if node_name in targets:
            connections.add(tuple(sorted([client, node_name])))
            
    return len(connections)

try:
    # Add each client node to the slice
    for node_name in client_node_names:
        print(f'> Processing client {node_name}')
        current_node = experiment_slice.add_node(
            name=node_name,
            site=experiment_sites[next(site_selector) % len(experiment_sites)],
            cores=experiment_cores,
            ram=experiment_ram,
            disk=experiment_disk,
            image=experiment_image
        ) 
        
        # Calculate total needed interfaces for this client
        total_interfaces = count_total_connections(node_name)
        print(f'> {node_name} needs {total_interfaces} interfaces')
        
        # Add all needed interfaces to the node
        for i in range(total_interfaces):
            print(f'> \tAdding interface {i+1} to client {node_name}')
            new_connection = current_node.add_component(
                model='NIC_Basic', 
                name=f'nic-{node_name[-1]}-{i+1}'
            ).get_interfaces()[0]
            
            client_interfaces[node_name].append(new_connection)
        
        client_nodes[node_name] = current_node
        
    # Add each general node to the slice
    for node_name in general_node_names:
        print(f'> Processing {node_name}')
        
        current_node = experiment_slice.add_node(
            name=node_name,
            site=experiment_sites[next(site_selector) % len(experiment_sites)],
            cores=experiment_cores,
            ram=experiment_ram,
            disk=experiment_disk,
            image=experiment_image
        )
        
        # Calculate total needed interfaces for this node
        total_interfaces = count_total_connections(node_name)
        print(f'> {node_name} needs {total_interfaces} interfaces')
        
        # Add all needed interfaces to the node
        for i in range(total_interfaces):
            print(f'> \tAdding interface {i+1} to {node_name}')
            new_connection = current_node.add_component(
                model='NIC_Basic',
                name=f'nic-{node_name[-1]}-{i+1}'
            ).get_interfaces()[0]
            
            nodes_interfaces[node_name].append(new_connection)
            
        nodes[node_name] = current_node

    # Debug output
    print("\nInterface counts per node:")
    for node_name in client_nodes:
        print(f"{node_name}: {len(client_interfaces[node_name])} interfaces")
    for node_name in nodes:
        print(f"{node_name}: {len(nodes_interfaces[node_name])} interfaces")
    
    print("\nConnection counts per node:")
    for node_name in {**client_nodes, **nodes}:
        print(f"{node_name}: {count_total_connections(node_name)} connections")
    
except Exception as e:
    print(f"> Node configuration failed: \n{e}")
    traceback.print_exc(e)

> Processing client node01
> node01 needs 1 interfaces
> 	Adding interface 1 to client node01
> Processing client node06
> node06 needs 1 interfaces
> 	Adding interface 1 to client node06
> Processing client node11
> node11 needs 1 interfaces
> 	Adding interface 1 to client node11
> Processing node02
> node02 needs 3 interfaces
> 	Adding interface 1 to node02
> 	Adding interface 2 to node02
> 	Adding interface 3 to node02
> Processing node03
> node03 needs 3 interfaces
> 	Adding interface 1 to node03
> 	Adding interface 2 to node03
> 	Adding interface 3 to node03
> Processing node04
> node04 needs 3 interfaces
> 	Adding interface 1 to node04
> 	Adding interface 2 to node04
> 	Adding interface 3 to node04
> Processing node05
> node05 needs 2 interfaces
> 	Adding interface 1 to node05
> 	Adding interface 2 to node05
> Processing node07
> node07 needs 4 interfaces
> 	Adding interface 1 to node07
> 	Adding interface 2 to node07
> 	Adding interface 3 to node07
> 	Adding interface 4 to node0

## Add Networks

In [6]:
# Create copies of interface tables
client_interfaces_copy = {k: v[:] for k, v in client_interfaces.items()}
nodes_interfaces_copy = {k: v[:] for k, v in nodes_interfaces.items()}

# Helper function to check if nodes are already connected
def are_nodes_connected(node1, node2, networks):
    return any(
        f'net-{n1}-{n2}' in networks.get(node1, [])
        for n1 in [node1[-1], node2[-1]]
        for n2 in [node1[-1], node2[-1]]
    )

# Helper function to create network connection
def create_network_connection(source_node, target_node, source_interface, target_interface, networks):
    fixed_source_node_name = source_node[-2:] if int(source_node[-2:]) <= 50 else source_node[-1]
    fixed_target_node_name = target_node[-2:] if int(target_node[-2:]) <= 50 else target_node[-1]
    network_name = f'net-{fixed_source_node_name}-{fixed_target_node_name}'
    
    # Create L2 network
    experiment_slice.add_l2network(
        name=network_name,
        interfaces=[source_interface, target_interface]
    )
    
    # Record network connection
    networks[source_node].append(network_name)
    networks[target_node].append(network_name)

try:
    # --- Add networks for clients
    for node_name in client_node_names:
        for connection in client_connections[node_name]:
            # Check if these nodes are not already connected
            if not are_nodes_connected(node_name, connection, networks):
                
                # Check if interfaces are available for both nodes
                if (node_name in client_interfaces_copy and client_interfaces_copy[node_name] and
                    connection in nodes_interfaces_copy and nodes_interfaces_copy[connection]):
                    
                    # Get interfaces for the connection
                    source_interface = client_interfaces_copy[node_name][0]
                    target_interface = nodes_interfaces_copy[connection][0]
                    
                    create_network_connection(
                        node_name, connection, 
                        source_interface, target_interface, 
                        networks
                    )
                    
                    # Remove used interfaces from copies
                    client_interfaces_copy[node_name].remove(source_interface)
                    nodes_interfaces_copy[connection].remove(target_interface)

    # --- Add networks for general nodes
    # Process all connections from general_connections
    processed_pairs = set()
    
    for source_node, target_nodes in general_connections.items():
        for target_node in target_nodes:
            # Create a sorted pair to avoid duplicates
            node_pair = tuple(sorted([source_node, target_node]))
            
            # Skip if this pair has been processed
            if node_pair in processed_pairs:
                continue
                
            # Check if these nodes are not already connected
            if not are_nodes_connected(source_node, target_node, networks):
                
                # Check if interfaces are available for both nodes
                if (source_node in nodes_interfaces_copy and nodes_interfaces_copy[source_node] and
                    target_node in nodes_interfaces_copy and nodes_interfaces_copy[target_node]):
                    
                    # Get interfaces for the connection
                    source_interface = nodes_interfaces_copy[source_node][0]
                    target_interface = nodes_interfaces_copy[target_node][0]
                    
                    create_network_connection(
                        source_node, target_node, 
                        source_interface, target_interface, 
                        networks
                    )
                    
                    # Remove used interfaces from copies
                    nodes_interfaces_copy[source_node].remove(source_interface)
                    nodes_interfaces_copy[target_node].remove(target_interface)
            
            # Mark this pair as processed
            processed_pairs.add(node_pair)

except Exception as e:
    traceback.print_exc(e)

# Print debug information
print("Final networks:", dict(networks))
print("Remaining interfaces for nodes:", dict(nodes_interfaces_copy))
print("Remaining interfaces for clients:", dict(client_interfaces_copy))

Final networks: {'node01': ['net-01-02'], 'node02': ['net-01-02', 'net-02-03', 'net-02-04'], 'node06': ['net-06-07'], 'node07': ['net-06-07', 'net-03-07', 'net-07-08', 'net-07-13'], 'node11': ['net-11-12'], 'node12': ['net-11-12', 'net-12-13', 'net-12-16'], 'node03': ['net-02-03', 'net-03-05', 'net-03-07'], 'node04': ['net-02-04', 'net-04-08', 'net-04-09'], 'node05': ['net-03-05', 'net-05-10'], 'node08': ['net-04-08', 'net-07-08', 'net-08-14'], 'node09': ['net-04-09', 'net-09-10', 'net-09-15'], 'node10': ['net-05-10', 'net-09-10', 'net-10-16'], 'node13': ['net-07-13', 'net-12-13', 'net-13-17'], 'node14': ['net-08-14', 'net-14-15', 'net-14-18'], 'node15': ['net-09-15', 'net-14-15', 'net-15-19'], 'node16': ['net-10-16', 'net-12-16', 'net-16-20'], 'node17': ['net-13-17', 'net-17-18'], 'node18': ['net-14-18', 'net-17-18', 'net-18-19'], 'node19': ['net-15-19', 'net-18-19', 'net-19-20'], 'node20': ['net-16-20', 'net-19-20']}
Remaining interfaces for nodes: {'node02': [], 'node03': [], 'node0

## Print information about nodes and connections

In [7]:
print(f'Nodes above. {pretty_print_defaultdict(nodes)}\n')
print(f'Nodes interfaces above. {pretty_print_defaultdict(nodes_interfaces)}\n')
print(f'Client nodes above. {pretty_print_defaultdict(client_nodes)}\n')
print(f'Client nodes interfaces above. {pretty_print_defaultdict(client_interfaces)}\n')
print(f'Networks above. {pretty_print_defaultdict(networks)}\n')

{   'node02': <fabrictestbed_extensions.fablib.node.Node object at 0x7bf2e420c250>,
    'node03': <fabrictestbed_extensions.fablib.node.Node object at 0x7bf2e6ccbc10>,
    'node04': <fabrictestbed_extensions.fablib.node.Node object at 0x7bf2e4302dd0>,
    'node05': <fabrictestbed_extensions.fablib.node.Node object at 0x7bf2e40fc890>,
    'node07': <fabrictestbed_extensions.fablib.node.Node object at 0x7bf2cc3d34d0>,
    'node08': <fabrictestbed_extensions.fablib.node.Node object at 0x7bf2e442a650>,
    'node09': <fabrictestbed_extensions.fablib.node.Node object at 0x7bf2e4359690>,
    'node10': <fabrictestbed_extensions.fablib.node.Node object at 0x7bf2e432fad0>,
    'node12': <fabrictestbed_extensions.fablib.node.Node object at 0x7bf2e6e1c390>,
    'node13': <fabrictestbed_extensions.fablib.node.Node object at 0x7bf2e40b5ad0>,
    'node14': <fabrictestbed_extensions.fablib.node.Node object at 0x7bf2cc3d31d0>,
    'node15': <fabrictestbed_extensions.fablib.node.Node object at 0x7bf2cc4

## Submit slice (Wait until Stable)

In [ ]:
# Create a checkpoint and only execute the rest of the code if the slice is stable
try:
    experiment_slice.submit()
except Exception as e:
    print(f"Slice submission error: {e}")


Retry: 0, Time: 21 sec


ID,f3986cc4-5b5f-47f9-8a2a-3ba9d492034c
Name,hydra_auto_1732544527
Lease Expiration (UTC),2024-11-26 14:22:59 +0000
Lease Start (UTC),2024-11-25 14:22:59 +0000
Project ID,b3cffedd-ddb4-43ee-b57d-459b768e14ca
State,Configuring


ID,Name,Site,Type,State,Error
6b5bce47-6dcd-46e5-867d-1c9b54c8a48f,node01,DALL,node,Ticketed,
a8a8abb2-819b-4f21-8637-9bb38a1e17a4,node02,WASH,node,Ticketed,
b74a282e-6a5a-4047-8a6b-4344f8aa3b9d,node03,MICH,node,Ticketed,
6178543b-1c3f-4b42-8415-a38c1338b1f2,node04,DALL,node,Ticketed,
282b23fe-ca38-44ee-a3d7-a1e113df42a5,node05,HAWI,node,Ticketed,
1770e1d4-108d-422e-a8d4-f983c957268b,node06,HAWI,node,Ticketed,
48a4ee66-389f-4728-bc8f-05fc33022a98,node07,NCSA,node,Ticketed,
27088082-5fe3-41fe-9701-15b7cac47f88,node08,WASH,node,Ticketed,
a9fcb319-5dd9-4f58-b4af-c057262b647b,node09,MICH,node,Ticketed,
504c8e35-a93f-4afc-b0ba-9a8d3f132edc,node10,DALL,node,Ticketed,


## Install Hydra dependencies in each node

In [9]:
# For each node, install all the dependencies

try:
    for node in experiment_slice.get_nodes():
        stdout, stderr = node.execute('sudo apt install software-properties-common')
        stdout, stderr = node.execute('sudo add-apt-repository ppa:named-data/ppa')
        stdout, stderr = node.execute('sudo apt update')
        stdout, stderr = node.execute('sudo apt install nfd')
        stdout, stderr = node.execute('sudo apt -y install python3-pip libndn-cxx-dev nfd ndnping ndnpeek ndn-dissect ndnchunks ndnsec')
        stdout, stderr = node.execute('git clone https://github.com/tntech-ngin/ndn-hydra.git')
        stdout, stderr = node.execute('cd ndn-hydra/ && pip install -e .')
        stdout, stderr = node.execute('pip3 install python-ndn ndn-storage ndn-svs numpy')
        stdout, stderr = node.execute('sudo apt install net-tools')
        stdout, stderr = node.execute('sudo apt update && sudo apt upgrade -y')
        stdout, stderr = node.execute('sudo cp /etc/ndn/nfd.conf.sample /etc/ndn/nfd.conf')
except Exception as e:
    print(f"Exception: {e}")

Reading package lists...
 

 Building dependency tree...
Reading state information...
software-properties-common is already the newest version (0.99.9.12).
software-properties-common set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
 A collection of binary packages related to NDN project
 More info: https://launchpad.net/~named-data/+archive/ubuntu/ppa
Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]
Get:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease [265 kB]
Get:3 http://ppa.launchpad.net/named-data/ppa/ubuntu focal InRelease [18.3 kB]
Get:4 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [3303 kB]
Get:5 http://ppa.launchpad.net/named-data/ppa/ubuntu focal/main amd64 Packages [6952 B]
Get:6 http://security.ubuntu.com/ubuntu focal-security/main Translation-en [484 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security/main amd64 c-n-f Metadata [14.3 kB]
Get:8 http://ppa.launchpad.n

## NFD environment configuration for each node

In [11]:
# Bring up the interfaces

for node in experiment_slice.get_nodes():
    for interface in node.get_interfaces():
        stdout, stderr = node.execute(f'sudo ip link set dev {interface.get_device_name()} up')
        stdout, stderr = node.execute('sleep 2; ip a')

all_node_names = client_node_names + general_node_names

for node_name in all_node_names:
    node = experiment_slice.get_node(node_name)
    stdout, stderr = node.execute('nfd-stop')
    stdout, stderr = node.execute('nfd-start')
    stdout, stderr = node.execute('sleep 2; nfd-status')
    stdout, stderr = node.execute(f'ndnsec key-gen /hydra/{node_name} | ndnsec cert-install -')

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host 
       valid_lft forever preferred_lft forever
2: enp3s0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 9000 qdisc fq_codel state UP group default qlen 1000
    link/ether fa:16:3e:ab:41:3d brd ff:ff:ff:ff:ff:ff
    inet 10.30.6.212/23 metric 100 brd 10.30.7.255 scope global dynamic enp3s0
       valid_lft 83380sec preferred_lft 83380sec
    inet6 2001:610:2d0:fabc:f816:3eff:feab:413d/64 scope global dynamic mngtmpaddr noprefixroute 
       valid_lft 86347sec preferred_lft 14347sec
    inet6 fe80::f816:3eff:feab:413d/64 scope link 
       valid_lft forever preferred_lft forever
3: enp7s0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 3a:8c:da:61:48:8c brd ff:ff:ff:ff:ff:ff
    inet6 

## Configure NFD faces and routes

In [13]:
client_interfaces_copy = defaultdict(list)

for client_node_name in client_node_names:
    client_node = experiment_slice.get_node(client_node_name)
    for interface in client_node.get_interfaces():
        client_interfaces_copy[client_node_name].append(interface)
        
nodes_interfaces_copy = defaultdict(list)
for node_name in general_node_names:
    node = experiment_slice.get_node(node_name)
    for interface in node.get_interfaces():
        nodes_interfaces_copy[node_name].append(interface)

nfd_connections = defaultdict(list)

def are_nodes_nfd_connected(source_node: str, target_node: str, all_nfd_connections: dict[str, list[str]]):
    return target_node in all_nfd_connections.get(source_node, [])

try:      
    for client_node_name in client_node_names:
        for connection in client_connections[client_node_name]:
            source_node = experiment_slice.get_node(client_node_name)
            
            # Check if these nodes are not already connected
            if not are_nodes_nfd_connected(client_node_name, connection, nfd_connections):
                print(f'NFD Connecting {client_node_name} to {connection}...')
                
                # Check if interfaces are available for both nodes
                if (client_node_name in client_interfaces_copy and 
                        client_interfaces_copy[client_node_name] and 
                        connection in nodes_interfaces_copy and nodes_interfaces_copy[connection]):
                            
                    # Get interfaces for the connection
                    source_node_interface = client_interfaces_copy[client_node_name][0]
                    target_node_interface = nodes_interfaces_copy[connection][0]
                    
                    print(f'Source node interface: {source_node_interface}')
                    print(f'Target node interface: {target_node_interface}')
                    
                    target_interface_mac_address, local_nic_name = "ether://[" + target_node_interface.get_mac() + "]", "dev://" + source_node_interface.get_device_name()

                    stdout, stderr = source_node.execute(f'nfdc face create remote {target_interface_mac_address} local {local_nic_name}')
                    stdout, stderr = source_node.execute('sleep 2; nfd-status')
                    stdout, stderr = source_node.execute(f'nfdc route add / {target_interface_mac_address}')
                    stdout, stderr = source_node.execute('sleep 2; nfd-status')
                    stdout, stderr = source_node.execute(f'nfdc cs config capacity 0 admit off serve off')
                    
                    # Remove used interfaces from copies
                    client_interfaces_copy[client_node_name].remove(source_node_interface)
                    nodes_interfaces_copy[connection].remove(target_node_interface)
                    
except Exception as e:
    traceback.print_exc(e)
            
processed_nfd_pairs = set()
try:
    for source_node_name, target_nodes in general_connections.items():
        for target_node_name in target_nodes:
            print(f'NFD Connecting {source_node_name} to {target_node_name}...')
            
            # Create a sorted pair to avoid duplicates
            node_pair = tuple(sorted([source_node_name, target_node_name]))
            
            # Skip if this pair has been processed
            if node_pair in processed_nfd_pairs:
                continue
                
            # Check if these nodes are not already connected
            if not are_nodes_nfd_connected(source_node_name, target_node_name, nfd_connections):
                source_node = experiment_slice.get_node(source_node_name)
               
                # Check if interfaces are available for both nodes
                if (source_node_name in nodes_interfaces_copy and nodes_interfaces_copy[source_node_name] and
                            target_node_name in nodes_interfaces_copy and nodes_interfaces_copy[target_node_name]):
                    
                    source_node_interface = nodes_interfaces_copy[source_node_name][0]
                    target_node_interface = nodes_interfaces_copy[target_node_name][0]
                    
                    print(f'Source node interface: \n{source_node_interface}')
                    print(f'Target node interface: \n{target_node_interface}')
                    
                    # Perform connections
                    stdout, stderr = source_node.execute('nfdc strategy set /hydra/group /localhost/nfd/strategy/multicast')
                    
                    remote_mac_address, local_nic_name = "ether://[" + target_node_interface.get_mac() + "]", "dev://" + source_node_interface.get_device_name()

                    stdout, stderr = source_node.execute(f'nfdc face create remote {remote_mac_address} local {local_nic_name}')
                    stdout, stderr = source_node.execute('sleep 2; nfd-status')
                    stdout, stderr = source_node.execute(f'nfdc route add /hydra/{target_node_name} {remote_mac_address}')
                    stdout, stderr = source_node.execute('sleep 2; nfd-status')
                    stdout, stderr = source_node.execute(f'nfdc route add /{target_node_name} {remote_mac_address}')
                    stdout, stderr = source_node.execute('sleep 2; nfd-status')
                    stdout, stderr = source_node.execute(f'nfdc route add /hydra/group {remote_mac_address}')
                    
                    # Disable cache
                    stdout, stderr = source_node.execute('sleep 2; nfd-status')
                    stdout, stderr = source_node.execute(f'nfdc cs config capacity 0 admit off serve off')
                    
                    # Remove used interfaces from copies
                    nodes_interfaces_copy[source_node_name].remove(source_node_interface)
                    nodes_interfaces_copy[target_node_name].remove(target_node_interface)    
                    
                # Mark this pair as processed
                processed_nfd_pairs.add(node_pair)

except Exception as e:
    traceback.print_exc(e)

print(processed_nfd_pairs)

NFD Connecting node01 to node02...
Source node interface: ---------------  -------------------------
Name             node01-nic-1-1-p1
Network          net-01-02
Bandwidth        100
Mode             config
VLAN
MAC              3A:8C:DA:61:48:8C
Physical Device  enp7s0
Device           enp7s0
Address          fe80::388c:daff:fe61:488c
Numa Node        6
Switch Port      HundredGigE0/0/0/5
---------------  -------------------------
Target node interface: ---------------  -------------------------
Name             node02-nic-2-3-p1
Network          net-02-06
Bandwidth        100
Mode             config
VLAN
MAC              26:06:71:29:B2:D2
Physical Device  enp9s0
Device           enp9s0
Address          fe80::2406:71ff:fe29:b2d2
Numa Node        4
Switch Port      HundredGigE0/0/0/5
---------------  -------------------------
face-exists id=264 local=dev://enp7s0 remote=ether://[26:06:71:29:b2:d2] persistency=persistent reliability=off congestion-marking=off congestion-marking-interva

## Get and Renew the slice

In [ ]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

slice_name = ''

#Set end host to now plus 14 days
end_date = (datetime.now(timezone.utc) + timedelta(days=14)).strftime("%Y-%m-%d %H:%M:%S %z")
try:
    current_slice = fablib.get_slice(name=slice_name)
    current_slice.renew(end_date)
except Exception as e:
    print(f"Exception: {e}")

## Check new expiration date

In [ ]:
try:
    current_slice = fablib.get_slice(name=slice_name)
    print(f"Lease End (UTC): {slice.get_lease_end()}")
       
except Exception as e:
    print(f"Exception: {e}")